# Ejercicio 6: Dense Retrieval e Introducción a FAISS
# Autor: Christian Pérez

## Objetivo de la práctica

Generar embeddings con sentence-transformers (SBERT, E5), e indexar documentos con FAISS

## Parte 0: Carga del Corpus
### Actividad

1. Carga el corpus 20 Newsgroups desde sklearn.datasets.fetch_20newsgroups.
2. Limita el corpus a los primeros 2000 documentos para facilitar el procesamiento.

In [1]:
from sklearn.datasets import fetch_20newsgroups
docs = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes')).data[:2000]

## Parte 2: Generación de Embeddings
### Actividad

1. Usa dos modelos de sentence-transformers. Puedes usar: `'all-MiniLM-L6-v2'` (SBERT), o `'intfloat/e5-base'` (E5). Cuando uses E5, antepon `"passage: "` a cada documento antes de codificar.
2. Genera los vectores de embeddings para todos los documentos usando el modelo seleccionado.
3. Guarda los embeddings en un array de NumPy para su posterior indexación.

In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
# Cargar el modelo SBERT
model = SentenceTransformer('all-MiniLM-L6-v2')
# Generar embeddings para todos los documentos
embeddings = model.encode(docs, convert_to_tensor=False, show_progress_bar=True)
# Convertir a array de NumPy y guardar
embeddings = np.array(embeddings)
print(f"Shape de embeddings: {embeddings.shape}")
print(f"Tipo de datos: {embeddings.dtype}")

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Shape de embeddings: (2000, 384)
Tipo de datos: float32


## Parte 3: Indexación con FAISS
### Actividad

1. Crea un índice plano con faiss.IndexFlatL2 para búsquedas por distancia euclidiana.
2. Asegúrate de usar la dimensión correcta `(embedding_dim = doc_embeddings.shape[1])`.
3. Agrega los vectores de documentos al índice.

In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 17.8 MB/s eta 0:00:00


In [ ]:
import faiss

# Obtener la dimensión de los embeddings
embedding_dim = embeddings.shape[1]
# Crear índice FAISS con distancia euclidiana (L2)
index = faiss.IndexFlatL2(embedding_dim)
# Agregar vectores al índice
index.add(embeddings.astype('float32'))

print(f"Dimensión del índice: {index.d}")
print(f"Número de vectores indexados: {index.ntotal}")
print(f"Índice entrenado: {index.is_trained}")

Dimensión del índice: 384
Número de vectores indexados: 2000
Índice entrenado: True


## Parte 4: Consulta Semántica
### Actividad

1. Escribe una consulta en lenguaje natural. Ejemplos:

    * "God, religion, and spirituality"
    * "space exploration"
    * "car maintenance"

2. Codifica la consulta utilizando el mismo modelo de embeddings. Cuando uses E5, antepon `"query: "` a la consulta.
3. Recupera los 5 documentos más relevantes con `index.search(...)`.
4. Muestra los textos de los documentos recuperados (puedes mostrar solo los primeros 500 caracteres de cada uno).

In [7]:
# Definir consulta
query = "space exploration"

# Codificar la consulta usando el mismo modelo
query_embedding = model.encode([query], convert_to_tensor=False)
query_embedding = np.array(query_embedding).astype('float32')

# Buscar los 5 documentos más relevantes
k = 5
distances, indices = index.search(query_embedding, k)

# Mostrar resultados
print(f"Consulta: '{query}'")
print(f"Documentos más relevantes:\n")

for i, (idx, dist) in enumerate(zip(indices[0], distances[0])):
    print(f"Documento {i+1} (índice {idx}, distancia: {dist:.4f}):")
    print(f"{docs[idx][:500]}...")
    print("-" * 80)

Consulta: 'space exploration'
Documentos más relevantes:

Documento 1 (índice 495, distancia: 1.0018):
I am posting this for a friend without internet access. Please inquire
to the phone number and address listed.
---------------------------------------------------------------------

"Space: Teaching's Newest Frontier"
Sponsored by the Planetary Studies Foundation

The Planetary Studies Foundation is sponsoring a one week class for
teachers called "Space: Teaching's Newest Frontier." The class will be
held at the Sheraton Suites in Elk Grove, Illinois from June 14 through
June 18. Participants wh...
--------------------------------------------------------------------------------
Documento 2 (índice 1643, distancia: 1.1204):

Well, here goes.

The first item of business is to establish the importance space life
sciences in the whole of scheme of humankind.  I mean compared
to football and baseball, the average joe schmoe doesn't seem interested
or even curious about spaceflight.  I thin